In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

In [2]:
data = pd.read_csv('SBERP_prepared.csv').dropna()

In [3]:
pd.options.mode.chained_assignment = None 
data['TIME1'] = data['TIME1'] / 10000
data['TIME2'] = data['TIME2'] / 10000
data['TIME3'] = data['TIME3'] / 10000
data['TIME4'] = data['TIME4'] / 10000
data['TIME5'] = data['TIME5'] / 10000
data['TIME6'] = data['TIME6'] / 10000
data['TIME7'] = data['TIME7'] / 10000
data['TIME8'] = data['TIME8'] / 10000
data['TIME9'] = data['TIME9'] / 10000

data.head()

,DATE,TIME1,OPEN1,HIGH1,LOW1,CLOSE1,VOL1,TIME2,OPEN2,HIGH2,...,HIGH8,LOW8,CLOSE8,VOL8,TIME9,OPEN9,HIGH9,LOW9,CLOSE9,VOL9
0,20120301,11.0,75.38,75.45,74.91,74.95,2334800,12.0,74.95,75.21,...,75.30,75.12,75.16,754600,19.0,75.14,75.61,75.12,75.58,2269200.0
1,20120302,11.0,76.01,76.35,75.79,76.13,3527600,12.0,76.14,76.30,...,76.72,76.03,76.72,4305900,19.0,76.72,77.48,76.68,77.09,6021500.0
2,20120305,11.0,77.09,78.59,77.09,78.39,5464300,12.0,78.38,78.40,...,78.11,77.72,78.10,1385100,19.0,78.10,78.11,77.53,77.59,2659900.0
3,20120306,11.0,77.15,77.20,76.71,77.03,2307800,12.0,77.03,77.11,...,75.83,75.03,75.07,1457600,19.0,75.07,75.11,73.80,73.81,5285200.0
4,20120307,11.0,74.00,74.20,73.23,73.86,3703100,12.0,73.86,74.37,...,74.56,73.90,74.40,2827700,19.0,74.40,74.78,74.21,74.53,5004100.0


In [4]:
from sklearn.utils import shuffle
data = shuffle(data)

## Задача на регрессию:
### Будет ли цена закрытия больше цены открытия ?

In [5]:
data['TARGET'] = data['CLOSE9'] > data['OPEN9']
data.head()

,DATE,TIME1,OPEN1,HIGH1,LOW1,CLOSE1,VOL1,TIME2,OPEN2,HIGH2,...,LOW8,CLOSE8,VOL8,TIME9,OPEN9,HIGH9,LOW9,CLOSE9,VOL9,TARGET
334,20130701,11.0,69.60,70.16,69.60,70.11,482300,12.0,70.11,70.50,...,70.77,71.16,872700,19.0,71.17,71.36,71.03,71.25,680600.0,True
387,20130912,11.0,74.99,75.28,74.77,74.95,878200,12.0,74.94,75.16,...,74.38,74.80,1713400,19.0,74.79,74.98,74.65,74.71,1056600.0,False
376,20130828,11.0,70.09,70.84,69.20,69.50,1885300,12.0,69.50,70.18,...,69.84,70.10,1138400,19.0,70.08,70.26,69.56,69.56,1004200.0,False
1264,20170314,11.0,118.61,119.01,117.68,118.27,555700,12.0,118.24,118.58,...,117.35,117.37,582900,19.0,117.35,118.10,117.21,117.51,847500.0,True
891,20150918,11.0,56.45,56.89,56.43,56.65,1136500,12.0,56.65,56.68,...,55.61,56.17,879400,19.0,56.18,56.38,56.00,56.37,868400.0,True


In [6]:
train_data = data[:1000]
test_data = data[1000:]

In [13]:
train_features = train_data.drop(["DATE", "CLOSE9", "LOW9", "HIGH9", "VOL9"], axis=1)
train_target = train_data["TARGET"]

test_features = test_data.drop(["DATE", "CLOSE9", "LOW9", "HIGH9", "VOL9"], axis=1)
test_target = test_data["TARGET"]

In [15]:
scaler = StandardScaler()

scaler.fit(train_features)
train_features_scaled = scaler.transform(train_features)

In [16]:
lr = LogisticRegression()
lr.fit(train_features_scaled, train_target)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [19]:
test_features_scaled = scaler.transform(test_features)
predictions = lr.predict(test_features)

ac = accuracy_score(test_target, predictions)
ps = precision_score(test_target, predictions)
rs = recall_score(test_target, predictions)
f1 = f1_score(test_target, predictions)
ra = roc_auc_score(test_target, predictions)

print("Доля правильных ответов:\t{}\nТочность\t\t\t{}\nПолнота\t\t\t{}\nF1 score\t\t{}\nROC\t\t\t{}".format(ac, ps, rs, f1, ra))

Доля правильных ответов:	0.477001703578
Точность			0.612903225806
Полнота			0.0605095541401
F1 score		0.110144927536
ROC			0.508276755092


In [20]:
res = pd.DataFrame(test_data["CLOSE9"].tolist(), columns = ["ACTUAL_CLOSE"])
res["OPEN"] = pd.Series(test_features["OPEN9"].tolist())
res["PREDICTED_CLOSE"] = pd.Series(predictions)
res["RESULT"] = np.where(res['PREDICTED_CLOSE'] == True, 
                         res['ACTUAL_CLOSE'] - res['OPEN'], 0)

print(np.sum(res['RESULT']))

2.420000000000009
